In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#settings


In [8]:
import subprocess
import sys
import pandas as pd
import os


varDataset='Privamov'  # the dataset name
varOriginalDatasetPath='/content/drive/My Drive/Mobility datasets/Privamov/'
varProjectPath="/content/drive/My Drive/Mobility datasets/Experiment/" # project path , where we will create our project
varTrainDays=15 # after trainDays days we begin the daily training ( federated training )
varHourOfSplit='23:59:59' # the time when we do the train
varFrequence=48 # training frequence, in 30min use case it is equal to 48 because 48*30min = 24h => one train each day

#Here we create a csv file which contains path files , lppm names,labels,parameters and colors ( colors are only needed to plot results)
data=[]
# NOBF case ***** important we have to keep the NOBF case as first case ***
varLppmNOBF='NOBF'
path=varOriginalDatasetPath+'/Train/NOBF-train-1800.csv' #have to be the first !!!
pathTest=varOriginalDatasetPath+'/Test/NOBF-test-1800.csv'
varNobfParameter='NOBF-1800'
label='NOBF'
color='#FF9700'
colorLight='#EE981B'
data.append([varDataset,varLppmNOBF,varNobfParameter,label,color,colorLight,path,pathTest])

# Geo-I cases
lppm ='Geo-I'
path=varOriginalDatasetPath+'/Train/geoi-train-D48-split--level-15-epsilon-001.csv'
pathTest=varOriginalDatasetPath+'/Test/geoi-test-D48-split--level-15-epsilon-001.csv'
param='epsilon-0001'
label='Geo-I 0.001'
color='#0C3F91'
colorLight='#1E55AD'
data.append([varDataset,lppm,param,label,color,colorLight,path,pathTest])

path=varOriginalDatasetPath+'/Train/geoi-train-D48-split--level-15-epsilon-005.csv'
pathTest=varOriginalDatasetPath+'/Test/geoi-test-D48-split--level-15-epsilon-005.csv'
param='epsilon-0005'
label='Geo-I 0.005'
color='#4981DC'
colorLight='#6D9CE6'
data.append([varDataset,lppm,param,label,color,colorLight,path,pathTest])

path=varOriginalDatasetPath+'/Train/geoi-train-D48-split--level-15-epsilon-01.csv'
pathTest=varOriginalDatasetPath+'/Test/geoi-test-D48-split--level-15-epsilon-01.csv'
param='epsilon-001'
label='Geo-I 0.001'
color='#6CCEFB'
colorLight='#97DCFD'
data.append([varDataset,lppm,param,label,color,colorLight,path,pathTest])


# Promesse cases
varLppmPROM='PROM'
path=varOriginalDatasetPath+'/Train/promesse-train-D48-split--level-15-distance-50.csv'
pathTest=varOriginalDatasetPath+'/Test/promesse-test-D48-split--level-15-distance-50.csv'
param='distance-50'
label='PROM 50m'
color='#1B891F'
colorLight='#33A03A'
data.append([varDataset,varLppmPROM,param,label,color,colorLight,path,pathTest])

path=varOriginalDatasetPath+'/Train/promesse-train-D48-split--level-15-distance-100.csv'
pathTest=varOriginalDatasetPath+'/Test/promesse-test-D48-split--level-15-distance-100.csv'
param='distance-100'
label='PROM 100m'
color='#34C739'
colorLight='#55D55D'
data.append([varDataset,varLppmPROM,param,label,color,colorLight,path,pathTest])

path=varOriginalDatasetPath+'/Train/promesse-train-D48-split--level-15-distance-200.csv'
pathTest=varOriginalDatasetPath+'/Test/promesse-test-D48-split--level-15-distance-200.csv'
param='distance-200'
label='PROM 200m'
color='#52F258'
colorLight='#7BF784'
data.append([varDataset,varLppmPROM,param,label,color,colorLight,path,pathTest])

#TRL Cases
lppm='TRL'
path=varOriginalDatasetPath+'/Train/tri-train-D48-split--level-15-range-1.csv'
pathTest=varOriginalDatasetPath+'/Test/tri-test-D48-split--level-15-range-1.csv'
param='range-1'
label='TRL 1km'
color='#B82C0F'
colorLight='#E04828'
data.append([varDataset,lppm,param,label,color,colorLight,path,pathTest])


path=varOriginalDatasetPath+'/Train/tri-train-D48-split--level-15-range-2.csv'
pathTest=varOriginalDatasetPath+'/Test/tri-test-D48-split--level-15-range-2.csv'
param='range-2'
label='TRL 2km'
color='#F5340C'
colorLight='#FD6241'
data.append([varDataset,lppm,param,label,color,colorLight,path,pathTest])

path=varOriginalDatasetPath+'/Train/tri-train-D48-split--level-15-range-3.csv'
pathTest=varOriginalDatasetPath+'/Test/tri-test-D48-split--level-15-range-3.csv'
param='range-3'
label='TRL 3km'
color='#FF6A6A'
colorLight='#FF8E8E'
data.append([varDataset,lppm,param,label,color,colorLight,path,pathTest])

# Create the pandas DataFrame
varDfInfoParameters = pd.DataFrame(data, columns = ['dataset','lppm','parameter','label','color','colorLight','pathTrain','pathTest'])
#dfInfoParameters.to_csv (parametersPath, index = None, header=True)


# Dataset preprocessing


1.   Convert dataType
2.   Uniform features
3.   Correct timestamps
4.   Reducing and centering
5.   Normalize ID



In [12]:
import pandas as pd
import os
import numpy as np
from sklearn.preprocessing import StandardScaler ,Binarizer,MaxAbsScaler,Normalizer,PowerTransformer,RobustScaler
import glob
from datetime import datetime 
from datetime import timedelta
import sys
from sklearn import preprocessing
import time

"""#Settings"""
dataset= varDataset # the dataset name
trainDays=varTrainDays# number of days to begin the daily training
originalDatasetPath=varOriginalDatasetPath# path to datasets
SavePath=varProjectPath# project path , where we will create our project
hourOfSplit=varHourOfSplit    # the time when we do the train
frequence=varFrequence # training frequence, in 30min use case it is equal to 48 because 48*30min = 24h => one train each day
dfInfoParameters = varDfInfoParameters
lppmNOBF=varLppmNOBF

# Global variables
allParameters=[]
lppmNames=[]
DatasetPath=SavePath+'/Prepared_Datasets/'+dataset+'/'

def convertToFloat32(df):
  IDs = df['IDs'].copy()
  timestamp=df['timestamp'].copy()
  del df['timestamp']
  del df['IDs']
  df=df.astype('float32')
  df['IDs']=IDs
  df['timestamp']=timestamp
  return df
def normalizeIDwithTable(df,table):
  df['oldIDs'] = df['IDs']
  df['IDs'] = df['IDs'].apply(lambda x: str(x).replace('-','_').split('_')[0])
  try : 
    df['IDs']=df ['IDs'].astype('int')
    df=df.sort_values(['IDs'])
  except : 
    print(" WARNING DATASET IS NOT INT TYPE \n")
  finally :  
    uniqueTrain=df['IDs'].unique()
    for uniqueID in uniqueTrain :
      df['IDs']= df['IDs'].replace(to_replace=uniqueID,value=tableNewOldID[uniqueID])
    return df
def normalizeID(train) :
  tableNewOldID = {}
  cptID=0
  train['oldIDs'] = train['IDs']
  train['IDs'] = train['IDs'].apply(lambda x: str(x).replace('-','_').split('_')[0])
  try : 
    train['IDs']=train ['IDs'].astype('int')
    train=train.sort_values(['IDs'])
  except : 
    print(" WARNING DATASET IS NOT INT TYPE \n")
  finally :  
    uniqueTrain=train['IDs'].unique()
    for uniqueID in uniqueTrain :
      train['IDs']= train['IDs'].replace(to_replace=uniqueID,value=cptID)
      tableNewOldID[uniqueID]=cptID
      cptID=cptID+1
    return [train,tableNewOldID]

def ReducingCentering(dfUnion) :
  dataUnion = dfUnion.values 
  sc = StandardScaler() 
  #transformation – centrage-réduction 
  dataNormalize = sc.fit_transform(dataUnion)
  dfUnion = pd.DataFrame(dataNormalize,columns=dfUnion.head(0).columns.values) 
  return dfUnion

def removeFeatures(df,featuresList):
  df=df.drop(featuresList, axis = 1) 
  return df

def removeAdditionalFeatures(df,featuresList,FeatureImportant):
  listToRemove=np.setdiff1d(featuresList, FeatureImportant)
  df=removeFeatures(df,listToRemove)
  return df

columnLists=[]
pathLists=[]

dfNobf=dfInfoParameters[dfInfoParameters['lppm']==lppmNOBF]
for idx,pathRow in dfNobf.iterrows() :
  pathLists.append(pathRow['pathTrain'])
  pathLists.append(pathRow['pathTest'])

#read headers only and got an union of all features
for pathFile in pathLists:
  columnList=pd.read_csv(pathFile, nrows=0).columns.tolist()
  columnLists=list(set().union(columnLists,columnList))


#add zeros and concat and save
for idx,dfRow in dfInfoParameters.iterrows():
  fileColumns=pd.read_csv(dfRow['pathTrain'], nrows=0).columns.tolist()
  ColumnsToRead=np.intersect1d(columnLists,fileColumns)
  dfTrain=pd.read_csv(dfRow['pathTrain'],usecols=ColumnsToRead)
  columnNames=dfTrain.columns.tolist()
  dfTrain=removeAdditionalFeatures(dfTrain,columnNames,columnLists)
  fileColumns=pd.read_csv(dfRow['pathTest'], nrows=0).columns.tolist()
  ColumnsToRead=np.intersect1d(columnLists,fileColumns)
  dfTest=pd.read_csv(dfRow['pathTest'],usecols=ColumnsToRead)
  columnNames=dfTest.columns.tolist()
  dfTest=removeAdditionalFeatures(dfTest,columnNames,columnLists)
  #concat train and test 
  df=pd.concat([dfTrain,dfTest])
  del dfTrain
  del dfTest
  print("Concatenation finished")
  df = df.fillna(0)
  df=df.reset_index(drop=True)
  
  #unify features
  df=convertToFloat32(df)
  print("converted to float 32")
  columnNames=df.columns.tolist()
  columnToAdd=list(set(columnLists) - set(columnNames))
  zeros=np.zeros(df.shape[0],dtype='uint8')
  for idx,column in enumerate(columnToAdd) :
    df[column]=zeros
  df = df.sort_index(axis=1)
  print("Uniform features finished")

  #correct timestamps (unify all timestamps with NOBF timestamps)
  lppm=dfRow['lppm']
  param=dfRow['parameter']
  if (lppm==lppmNOBF) :
    dfNOBF_timestamp_ids=df[['IDs','timestamp']].copy()
  else : 
    del df['timestamp']
    df=pd.merge(df, dfNOBF_timestamp_ids, how='inner', on=['IDs'])
    df.dropna(inplace=True)

  #Reducing and centering data
  IDs=df['IDs'].copy()
  total=df['total'].copy()
  Timestamps=df['timestamp'].copy()
  df.pop('timestamp')
  df.pop('IDs')
  df.pop('total') 
  df= ReducingCentering(df)
  df['IDs']=IDs
  df['timestamp']=Timestamps
  df['total']=total
  print(" Reduction centering finished")


  #transform the IDs format and store the old one's on oldIDs
  if(lppm==lppmNOBF) :
    [df,tableNewOldID]=normalizeID(df)
  else :
    df=normalizeIDwithTable(df,tableNewOldID)
  print(" format IDs finished")

  #save the formated dataset
  if not os.path.exists(DatasetPath):
    os.makedirs(DatasetPath)
  SaveDatasetPath=DatasetPath+dataset+'-'+lppm+'-'+param+'.csv'
  df.to_pickle(SaveDatasetPath, compression='zip')
  print("save",df.shape)

Concatenation finished
converted to float 32
Uniform features finished
 Reduction centering finished
 format IDs finished
save (6554, 5260)
Concatenation finished
converted to float 32
Uniform features finished
 Reduction centering finished
 format IDs finished
save (6554, 5260)
Concatenation finished
converted to float 32
Uniform features finished
 Reduction centering finished
 format IDs finished
save (6554, 5260)
Concatenation finished
converted to float 32
Uniform features finished
 Reduction centering finished
 format IDs finished
save (6554, 5260)
Concatenation finished
converted to float 32
Uniform features finished
 Reduction centering finished
 format IDs finished
save (4883, 5260)
Concatenation finished
converted to float 32
Uniform features finished
 Reduction centering finished
 format IDs finished
save (3200, 5260)
Concatenation finished
converted to float 32
Uniform features finished
 Reduction centering finished
 format IDs finished
save (1601, 5260)
Concatenation finish

# Generate DRk

In [13]:
"""# Generate DRk"""
import glob
import pandas as pd
from datetime import datetime 
from datetime import timedelta
import shutil
import os
import sys



trainDays=varTrainDays
dfInfoParameters=varDfInfoParameters
dataset=varDataset
ProjectPath=varProjectPath
hourOfSplit=varHourOfSplit
frequence=varFrequence
lppmNOBF=varLppmNOBF
DatasetPath=ProjectPath+'/Prepared_Datasets/'+dataset+'/'

def prepareDataset (datasetName,file,trainDays,dirDatasetTrainAfk,hourOfSplit='23:59:59',frequence=48) :
    if not os.path.exists(DatasetPath):
        os.makedirs(DatasetPath)
    if not os.path.exists(dirDatasetTrainAfk):
        os.makedirs(dirDatasetTrainAfk)
    dfUnion=pd.read_pickle(file,compression='zip')
    n_class=dfUnion['IDs'].nunique()
  
    #----------------Extract the train AF0/AC0-----------------------
    dfUnion=dfUnion.sort_values(['timestamp'])
    firstTraceDate = dfUnion.iloc[0]['timestamp'] 
    try : 
      firstTraceDate_obj = datetime.strptime(firstTraceDate, '%Y-%m-%d %H:%M:%S')
      splitDate = firstTraceDate_obj + timedelta(days=trainDays)  
      timestampSplit=splitDate.strftime('%Y-%m-%d %H:%M:%S')
    except : 
      print("timestamp format error")
      firstTraceDate_obj = datetime.strptime(firstTraceDate, '%Y-%m-%d %H:%M:%S.%f')
      splitDate = firstTraceDate_obj + timedelta(days=trainDays)  
      timestampSplit=splitDate.strftime('%Y-%m-%d %H:%M:%S.%f')
  

    #Split the dataframe dfUnion into training and testing set
    dfTest = dfUnion[dfUnion['timestamp']>timestampSplit]
    dfTrain = dfUnion[dfUnion['timestamp']<=timestampSplit]
    dfTest=dfTest.reset_index(drop=True)
    
    timestampSplit = splitDate.strftime('%Y-%m-%d')
    timestampSplit = timestampSplit +" "+hourOfSplit

    #Split the dataframe dfTest into DR1 and DRK k >1 to ajust time split
    dfDR1 = dfTest[dfTest['timestamp']<=timestampSplit]
    dfTest = dfTest[dfTest['timestamp']>timestampSplit]
    dfTest=dfTest.reset_index(drop=True)

  
    #save DR0 and IR0

    dfTrain= dfTrain.sort_values(['IDs'])
    InfoTrain=dfTrain[['IDs','oldIDs','timestamp']]
    InfoTrain.to_csv(dirDatasetTrainAfk+"/IR0.csv") 
    dfTrain.pop('oldIDs')
    dfTrain.pop('timestamp')
    dfTrain.to_pickle(dirDatasetTrainAfk+"/DR0.csv",compression='zip') 
    n_features = dfTrain.shape[1] 

    #save DR1 and IR1

    dfDR1= dfDR1.sort_values(['IDs'])
    InfoTrain=dfDR1[['IDs','oldIDs','timestamp']]
    InfoTrain.to_csv(dirDatasetTrainAfk+"/IR1.csv") 
    dfDR1.pop('oldIDs')
    dfDR1.pop('timestamp')
    dfDR1.to_pickle(dirDatasetTrainAfk+"/DR1.csv",compression='zip') 
    testFile=dfTest['oldIDs']

    #-----------------------------------------------generate DRk datasets (k > 0) -------------------------------------------------
    round = 1   
    traceList = ((testFile.apply(lambda x: str(x).replace('-','_').split('_')[1])).values).astype(int)
    currentTrace=traceList[0]
    traceNumberRound = 0
    for (idx,trace) in enumerate(traceList) :
      if((currentTrace+frequence)>trace) :
        traceNumberRound = traceNumberRound + 1
      else :
        round = round + 1
        df2Split = dfTest.loc[(idx-traceNumberRound):(idx-1),:]
        #saveTimesTamp and oldIDS
        df2Split= df2Split.sort_values(['IDs'])
        dfSplitInfo=df2Split[['IDs','oldIDs','timestamp']]
        dfSplitInfo.to_csv(dirDatasetTrainAfk+"/IR"+str(round)+".csv") 
        df2Split.pop('timestamp')
        df2Split.pop('oldIDs')
        df2Split.to_pickle (dirDatasetTrainAfk+"/DR"+str(round)+".csv",compression='zip') 
        traceNumberRound = 1 
        currentTrace=trace
    #Save the rest 
    round = round + 1
    df2Split = dfTest.loc[(idx-traceNumberRound):(idx-1),:]
    df2Split= df2Split.sort_values(['IDs'])
    timestamptestFile=df2Split[['IDs','oldIDs','timestamp']]
    timestamptestFile.to_csv(dirDatasetTrainAfk+"/IR"+str(round)+".csv") 
    df2Split.pop('timestamp')
    df2Split.pop('oldIDs')
    df2Split.to_pickle(dirDatasetTrainAfk+"/DR"+str(round)+".csv",compression='zip') 
    return [n_class,n_features]

allParameters=dfInfoParameters['parameter']
for param in allParameters :
  print("*************************************************")
  print(param)
  print(DatasetPath)
  fileToDevide=glob.glob(DatasetPath+'*'+param+'.csv')
  print(fileToDevide[0])
  directoryToSave=DatasetPath+'/'+param
  print("before preparedataset function")
  [n_class,n_features]=prepareDataset (dataset,fileToDevide[0],trainDays,directoryToSave,hourOfSplit=hourOfSplit,frequence=frequence) 
  print(n_class)
  print(n_features)
  if(param == dfInfoParameters[dfInfoParameters['lppm']==lppmNOBF]['parameter'].values.item()):
    DatasetInfoPath=DatasetPath+'/DatasetInfo.csv'
    dfInfo = pd.DataFrame(data=[[dataset,n_class,n_features]], columns = ['Dataset', 'classNumber','featureNumber']) 
    dfInfo.to_pickle (DatasetInfoPath,compression='zip')
    print(dfInfo)


'''Split each DRk per User'''

#separate in a DRk , users : 
inputDirectory=varProjectPath+"/"+"Prepared_Datasets/"+dataset+"/"+"NOBF-1800"
outputDirectory=varProjectPath+"/"+"Prepared_Datasets/"+dataset+"/"+"NOBF-1800-split"

#extract the number of rounds
path =inputDirectory+'/DR*.csv'
filesList=glob.glob(path)   
numberOfRound = len(filesList)

if not os.path.exists(outputDirectory):
	os.makedirs(outputDirectory)

df=pd.read_pickle(inputDirectory+'/DR0.csv',compression='zip')

for i in range(0,numberOfRound):
	print("round",i)
	df=pd.read_pickle(inputDirectory+'/DR'+str(i)+'.csv',compression='zip')
	ids=df['IDs'].tolist()
	ids_unique= list(set(ids))
	grouped = df.groupby(df.IDs)

	for user in ids_unique:
		user_df = grouped.get_group(user)
		user_df=user_df.reset_index(drop=True)
		if not os.path.exists(outputDirectory+'/DR'+str(i)+'/'):
			os.makedirs(outputDirectory+'/DR'+str(i)+'/')
		user_df.to_pickle(outputDirectory+'/DR'+str(i)+'/'+str(user)+'.csv',compression='zip')





    dataset  ...                                           pathTest
0  Privamov  ...  /content/drive/My Drive/Mobility datasets/Priv...
1  Privamov  ...  /content/drive/My Drive/Mobility datasets/Priv...
2  Privamov  ...  /content/drive/My Drive/Mobility datasets/Priv...
3  Privamov  ...  /content/drive/My Drive/Mobility datasets/Priv...
4  Privamov  ...  /content/drive/My Drive/Mobility datasets/Priv...
5  Privamov  ...  /content/drive/My Drive/Mobility datasets/Priv...
6  Privamov  ...  /content/drive/My Drive/Mobility datasets/Priv...
7  Privamov  ...  /content/drive/My Drive/Mobility datasets/Priv...
8  Privamov  ...  /content/drive/My Drive/Mobility datasets/Priv...
9  Privamov  ...  /content/drive/My Drive/Mobility datasets/Priv...

[10 rows x 8 columns]
*************************************************
NOBF-1800
/content/drive/My Drive/Mobility datasets/Experiment//Prepared_Datasets/Privamov/
/content/drive/My Drive/Mobility datasets/Experiment//Prepared_Datasets/Privamov/Privam

# Generate models


In [17]:
import pandas as pd
import numpy as np
from collections import Counter
import torch
from datetime import datetime
import torch.nn as nn
import time
import glob
import sys
import os
from torch.nn import functional as F


"""#Config Project

## Parameters
"""
dataset= varDataset # the dataset name
projectPath= varProjectPath #path where we have created our project
epoch= 1000# the maximum number of epochs we can do ( federated and cetralized version)
batch= 1 # batch size
lr= 0.001 # learning rate value for the simple gradient descent optimizer
Round = 0 # from wich round we begin the train ( if first time set Round = 0 )
#parametersPath = sys.argv[10]
dfInfoParameters=varDfInfoParameters
"""Global variables"""

#dfInfoParameters=pd.read_csv(parametersPath)
NOBFParameter=  dfInfoParameters[dfInfoParameters['lppm']=='NOBF']['parameter'].values.item()
allParameters=dfInfoParameters['parameter']
DatasetInfoPath=projectPath+'Prepared_Datasets/'+dataset+'/DatasetInfo.csv'
ResultAccuracyPath=projectPath+'/Evaluation/'+dataset+'/Results/'
ModelsPath = projectPath+'/Models/'+dataset+'/'
pathDRKC=projectPath+'Prepared_Datasets/'+dataset+'/NOBF-1800/'  
pathDRKF=projectPath+'Prepared_Datasets/'+dataset+'/NOBF-1800-split/'  

class Dataset(torch.utils.data.Dataset):
    'Characterizes a dataset for PyTorch'
    def __init__(self, list_IDs, labels):
        'Initialization'
        self.labels = labels
        self.list_IDs = list_IDs

    def __len__(self):
        'Denotes the total number of samples'
        return len(self.list_IDs)

    def __getitem__(self, index):
        # Load data and get label
        X = self.list_IDs[index]
        y = self.labels[index]
        return X, y

"""## Load dataset"""

def getInfo(datasetName,columnName) : 
  dfInfo = pd.read_pickle(DatasetInfoPath,compression='zip')
  dfInfo = dfInfo[dfInfo['Dataset'] == datasetName]
  info = dfInfo[columnName].values.item(0)
  return info


def loadDataset(datafile):
  df = pd.read_pickle(datafile,compression='zip')#.fillna(0)
  training_target = df['IDs'].values
  df.pop('IDs')
  factor=df['total'].values
  df.pop('total') # besma poping
  training_features=df.values
  training_features=np.asarray(training_features, dtype=np.float32)
  training_target = np.asarray(training_target, dtype=np.int16)
  return training_features, training_target,factor


#Create a data loader for  centralized dataset 
def createDataLoader(inputs,labels,train):
  inputs_torch = torch.tensor(inputs,dtype=torch.float, requires_grad=True).cuda()
  labels_torch = torch.tensor(labels,dtype=torch.long).cuda()
  Dataset_set = Dataset(inputs_torch, labels_torch)
  if train :
    loader = torch.utils.data.DataLoader(dataset=Dataset_set,batch_size=len(labels),shuffle=True)
  else :
    loader = torch.utils.data.DataLoader(dataset=Dataset_set,batch_size=1,shuffle=False)
  return loader

class LogisticRegressionModel(nn.Module):
  def __init__(self, input_dim, output_dim):
    """
    input_dim: number of input features.
    output_dim: number of labels.
    """
    super(LogisticRegressionModel, self).__init__()
    self.linear = torch.nn.Linear(input_dim, output_dim)
  def forward(self, x):
    outputs = self.linear(x)
    return outputs


def getModel(datasetName,round=0) : 
  n_feature = getInfo(datasetName,'featureNumber')-1-1 #remove labels column and total col
  n_class = getInfo(datasetName,'classNumber')
  model = LogisticRegressionModel(n_feature, n_class).cuda()
  if(round > 0) :
    modelPath = ModelsPath+'/AF'+str(round-1)+'.pt'
    model = torch.load(modelPath)
  return model

def local_train(model, device, federated_train_loader, optimizer):
  model.train()
  lossFunction = nn.CrossEntropyLoss() #weights
    # Iterate through each gateway's dataset
  for idx, (data, target) in enumerate(federated_train_loader):
      batch_idx = idx+1
      # Move the data and target labels to the device (cpu/gpu) for computation
      data, target = data.to(device), target.to(device)
      # Clear previous gradients (if they exist)
      optimizer.zero_grad()
      # Make a prediction
      output = model(data)
      loss= lossFunction(output,target)
      # Calculate the gradients
      loss.backward()
      # Update the model weights
      optimizer.step()
      # Get the model back from the gateway
  return [model,loss.item()]



def generateAFK(model,dataset,round,modelDirectory,resultDirectory, X_train, y_train,EPOCHS = 10,BATCH_SIZE =1,lr = 0.0001,firstUser=False) :  
  optimizer = torch.optim.SGD(model.parameters(), lr=lr)
  scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer, base_lr=0.0001, max_lr=0.01)
  epoch=0
  federated_train_loader=createDataLoader(X_train,y_train,True)
  
  while (epoch < EPOCHS):
      [model,loss]=local_train(model, device, federated_train_loader, optimizer) #,weights
      epoch=epoch +1
  model_copy=model
  return model_copy




# ************************************* Run Experimentation *************************************#
##################################################################################################

#here we precise if we will use cpu or cuda ( for cuda , we should add other modifications)
device = torch.device("cuda")
n_feature = getInfo(dataset,'featureNumber')-1-1 #remove labels column and total col
n_class = getInfo(dataset,'classNumber')



#extract the number of rounds
path =pathDRKF+'/DR*'
filesList=glob.glob(path)   
NumRounds = len(filesList)

modelF=getModel(dataset,Round)

if not os.path.exists(ModelsPath):
  os.makedirs(ModelsPath)

while (Round < NumRounds) :  
  print("-----------------------------------------------------------------------------------------")
  print("Round  :" + str(Round  ) ) 
  print("------------------------------------------------------------------------------------------")

  modelPathResult=ModelsPath+'AF'+str(Round)+'.pt'
  for idx,filename in enumerate(os.listdir(pathDRKF+'DR'+str(Round)+'/')):
    if idx==0: 
      firstUser=True
    else: 
      firstUser=False
    nb_current_users=len([name for name in os.listdir(pathDRKF+'DR'+str(Round)+'/') if os.path.isfile(os.path.join(pathDRKF+'DR'+str(Round)+'/', name))])
    user_name=filename.split('.')[0]
    trainFile=pathDRKF+'DR'+str(Round)+'/'+filename
    X_train, y_train,factor= loadDataset(trainFile)
    resultDirectory=ResultAccuracyPath+NOBFParameter+'/'            
    modelF=generateAFK(modelF,dataset,Round,modelPathResult,resultDirectory, X_train, y_train,EPOCHS = epoch,BATCH_SIZE =len(y_train),lr = lr,firstUser=firstUser)   
  torch.save(modelF,modelPathResult)
  Round = Round + 1

-----------------------------------------------------------------------------------------
Round  :0
------------------------------------------------------------------------------------------
-----------------------------------------------------------------------------------------
Round  :1
------------------------------------------------------------------------------------------
-----------------------------------------------------------------------------------------
Round  :2
------------------------------------------------------------------------------------------
-----------------------------------------------------------------------------------------
Round  :3
------------------------------------------------------------------------------------------
-----------------------------------------------------------------------------------------
Round  :4
------------------------------------------------------------------------------------------
---------------------------------------------

#Test models

In [23]:
import pandas as pd
import numpy as np
from collections import Counter
import torch
import torch.nn as nn
import sys
import glob
from torch.nn import functional as F

"""#Settings"""

dataset= varDataset
pathProject= varProjectPath
dfInfoParameters=varDfInfoParameters
NOBFParameters=varNobfParameter
promesseParameters=dfInfoParameters[dfInfoParameters['lppm']==varLppmPROM]['parameter'].values
pathDRKC=pathProject+'Prepared_Datasets/'+dataset+'/NOBF-1800/'
path =pathDRKC+'/DR*.csv'
filesList=glob.glob(path)   
nb_rounds = len(filesList)-1 

class LogisticRegressionModel(nn.Module):
  def __init__(self, input_dim, output_dim):
    """
    input_dim: number of input features.
    output_dim: number of labels.
    """
    super(LogisticRegressionModel, self).__init__()
    self.linear = torch.nn.Linear(input_dim, output_dim)
  def forward(self, x):
    outputs = self.linear(x)#F.softmax(x,dim=1)
    return outputs


class Dataset(torch.utils.data.Dataset):
    'Characterizes a dataset for PyTorch'

    def __init__(self, list_IDs, labels):
        'Initialization'
        self.labels = labels
        self.list_IDs = list_IDs

    def __len__(self):
        'Denotes the total number of samples'
        return len(self.list_IDs)

    def __getitem__(self, index):
        # Load data and get label
        X = self.list_IDs[index]
        y = self.labels[index]
        return X, y

def loadTest(testFile):
  datafile2 = testFile
  df2 = pd.read_pickle(datafile2,compression='zip')
  y2 = df2['IDs'].values
  df2.pop('IDs')
  df2.pop('total')  
  x2=df2.values
  x2=np.asarray(x2, dtype=np.float32)
  y2 = np.asarray(y2, dtype=np.float32)
  n_features= x2.shape[1]
  testing_features, testing_target  = x2, y2
  user_count_dict = Counter(testing_target)
  user_types = list(user_count_dict.keys())
  return len(user_types),n_features,testing_features, testing_target

def loadModel(n_feature,n_class,pathmodel):
  model_new = LogisticRegressionModel(n_feature,n_class).cuda()
  model_new=torch.load(pathmodel) 
  return model_new

#Create a data loader for  centralized dataset 
def createDataLoader(inputs,labels):
  inputs_torch = torch.tensor(inputs,dtype=torch.float, requires_grad=True).cuda()
  labels_torch = torch.tensor(labels,dtype=torch.long).cuda()
  Dataset_set = Dataset(inputs_torch, labels_torch)
  loader = torch.utils.data.DataLoader(dataset=Dataset_set,batch_size=1,shuffle=False)
  return loader


def testdecentralized(model,test_loader):
    model.eval()
    correct = 0
    total = 0
    df = pd.DataFrame(columns=['target','prediction','origin-target'])
    for data, target in test_loader:
          # Forward pass only to get logits/output
          outputs = model(data)
          # Get predictions from the maximum value
          _, predicted = torch.max(outputs.data, 1)
          # Total number of labels
          total += target.size(0)
          # Total correct predictions
          correct += (predicted == target).sum()  
          #Save prediction in csv file 
          predicted=predicted.item()
          target=target.item()  
          df = df.append({'target':target, 'prediction':predicted}, ignore_index=True)
    return correct.item(),total,df

def concat_save(df_pred,df_IR,pathPrediction):
  #concatener IRk et pred-k
  df_pred['origin-target']=df_IR['oldIDs']
  df_pred.to_csv(pathPrediction, index=False)

def result_save(df,pathResult):
  df.to_csv(pathResult, index=False)

#**************************  Load data ***********************************#

dfInfo = pd.read_pickle(pathProject+'Prepared_Datasets/'+dataset+'/DatasetInfo.csv',compression='zip')
nb_users = dfInfo[dfInfo['Dataset'] == dataset]['classNumber'].values.item(0)
i=0
df = pd.DataFrame(columns=['Dataset','Lppm','Model','Round','Nb_user','Correct','Total','Accuracy'])
pathResult=pathProject+'/Evaluation/'+dataset+'/Result_final_'+dataset+'.csv'
totalnobf=[]
for lppm in dfInfoParameters['parameter']:
  directory=pathProject+'/Evaluation/'+dataset+'/Predictions/'+lppm
  if not os.path.exists(directory):
    os.makedirs(directory)
  for nb_round in range(0,nb_rounds):
    print("round: ", nb_round)
    testFile=pathProject+'/Prepared_Datasets/'+dataset+'/'+lppm+'/DR'+str(nb_round+1)+'.csv'  
    pathIRk=pathProject+'/Prepared_Datasets/'+dataset+'/'+lppm+'/IR'+str(nb_round+1)+'.csv'
    pathPrediction2=pathProject+'/Evaluation/'+dataset+'/Predictions/'+lppm+'/prediction-decentralized-'+str(nb_round+1)+'.csv' 
    pathmodel_decentralized=pathProject+'/Models/'+dataset+'/AF'+str(nb_round)+'.pt'  # Running : 
    #*******************************************************************************#
    # Test federated model with simulation 
    #*******************************************************************************#
    nb_users_test, n_features, X_test, y_test = loadTest(testFile)
    if lppm=='NOBF-1800':
      totalnobf.append(len(y_test))
    #2- Load the federated model AF
    federated_model= loadModel(n_features,nb_users,pathmodel_decentralized) 
    test_loader=createDataLoader(X_test,y_test)
    correct2,total2,df_pred2=testdecentralized(federated_model,test_loader)
    print(lppm,dataset)
    print("decentralized accuracy", correct2,totalnobf[nb_round],correct2/totalnobf[nb_round])
    #3- Prepare prediction file
    df_IR = pd.read_csv(pathIRk)
    concat_save(df_pred2,df_IR,pathPrediction2)
    #'Dataset','Lppm','Model','Round','Nb_user','Correct','Total','Accuracy'])
    df = df.append({'Dataset':dataset, 'Lppm':lppm,'Model':'AF'+str(nb_round),'Round':str(nb_round+1),'Nb_user':nb_users_test,'Correct':correct2,'Total':totalnobf[nb_round],'Accuracy':correct2/totalnobf[nb_round]}, ignore_index=True)
   
result_save(df,pathResult)

round:  0
NOBF-1800 Privamov
decentralized accuracy 137 189 0.7248677248677249
round:  1
NOBF-1800 Privamov
decentralized accuracy 200 210 0.9523809523809523
round:  2
NOBF-1800 Privamov
decentralized accuracy 204 247 0.8259109311740891
round:  3
NOBF-1800 Privamov
decentralized accuracy 84 132 0.6363636363636364
round:  4
NOBF-1800 Privamov
decentralized accuracy 48 51 0.9411764705882353
round:  5
NOBF-1800 Privamov
decentralized accuracy 105 137 0.7664233576642335
round:  6
NOBF-1800 Privamov
decentralized accuracy 125 133 0.9398496240601504
round:  7
NOBF-1800 Privamov
decentralized accuracy 138 205 0.6731707317073171
round:  8
NOBF-1800 Privamov
decentralized accuracy 195 287 0.6794425087108014
round:  9
NOBF-1800 Privamov
decentralized accuracy 116 209 0.5550239234449761
round:  10
NOBF-1800 Privamov
decentralized accuracy 98 139 0.7050359712230215
round:  11
NOBF-1800 Privamov
decentralized accuracy 55 137 0.40145985401459855
round:  12
NOBF-1800 Privamov
decentralized accuracy 1

KeyboardInterrupt: ignored